In [3]:
# 우리가 실제로 데이터 다루다보면 대용량인 경우가 많다
# 대용량은 두 가지 관점에서 볼 수 있다. 데이터 종류가 많은 것과 개수가 많은 것
# 먼저 데이터 종류가 많은 경우, 그게 특징 선택 방법
    # 다 쓰지 않고 일부만 골라내서 쓴다
# 물론 특징이 많아도 돌아가는 경우가 있다
    # 대표적으로 나이브베이즈 모델.
# 그런데 서포트벡터머신은 데이터 종류가 많으면 완전 쥐약이다. 제곱에 비례해 계산량이 늘어나기 때문에
# 랜덤 포레스트, 디시전 트리, 부스팅은 선형적으로 시간이 늘어난다. 내가 조사해야할 피처의 개수가 늘어나니까
# 그래서 아예 돌리지 못하는 경우가 생기기도 한다

# 특징 선택

실무에서는 대규모의 데이터를 기반으로 분류예측 모형을 만들어야 하는 경우가 많다. 대규모의 데이터라고 하면 표본의 갯수가 많거나 아니면 독립변수 즉, 특징데이터의 종류가 많거나 혹은 이 두가지 모두인 경우가 있다. 여기에서는 특징데이터의 종류가 많은 경우에 가장 중요하다고 생각되는 특징데이터만 선택하여 특징데이터의 종류를 줄이기 위한 방법에 대해 알아본다.

In [1]:
%%time
from sklearn.datasets import fetch_rcv1
rcv_train = fetch_rcv1(subset="train")
rcv_test = fetch_rcv1(subset="test")
X_train = rcv_train.data
y_train = rcv_train.target
X_test = rcv_test.data
y_test = rcv_test.target

# Ont-Hot-Encoding된 라벨을 정수형으로 복원
classes = np.arange(rcv_train.target.shape[1])
y_train = y_train.dot(classes)
y_test = y_test.dot(classes)

print(X_train.shape)

(23149, 47236)
CPU times: user 7.99 s, sys: 1.14 s, total: 9.13 s
Wall time: 9.15 s


In [10]:
# 데이터 개수 2만3000개, 데이터 종류 4만7000개

## 분산에 의한 선택

In [8]:
# 1. 먼저 x가 움지이지 않고 있는 애들이 있으면 빼는 게 좋다.
    # y값이 달라지면 x값도 달라져야 그 다음에 상관관계가 있고 없고를 판단한다
    # 그런데 x 값이 아예 변하지 않고 있으면 상관관계라는 게 존재하지 않는다
    # 안 움직이는지 움직이는지는 분산(variance), 표준편차으로 판단한다. 둘이 내가 정해놓은 기준치보다 적은 애들은 뺀다
# 2. 특징 데이터가 모두 0아니면 1이다, 이런 애들 중에서 특징 데이터가 한 쪽으로 쏠려 있는 경우 x 빼는 게 좋다

# 상관관계랑 일치한다는 보장은 없다. 경험적으로 괜찮다는 것. 무엇보다 이걸 테스트하는 데 들어가는 계산량이 적다

원래 예측모형에서 중요한 특징데이터란 종속데이터와의 상관관계가 크고 예측에 도움이 되는 데이터를 말한다. 하지만 상관관계 계산에 앞서 특징데이터의 값 자체가 표본에 따라 그다지 변하지 않는다면 종속데이터 예측에도 도움이 되지 않을 가능성이 높다. 따라서 표본 변화에 따른 데이터 값의 변화 즉, 분산이 기준치보다 낮은 특징 데이터는 사용하지 않는 방법이 분산에 의한 선택 방법이다.
예를 들어 종속데이터와 특징데이터가 모두 0 또는 1 두가지 값만 가지는데 종속데이터는 0과 1이 균형을 이루는데 반해 특징데이터가 대부분(예를 들어 90%)의 값이 0이라면 이 특징데이터는 분류에 도움이 되지 않을 가능성이 높다.

하지만 분산에 의한 선택은 반드시 상관관계와 일치한다는 보장이 없기 때문에 신중하게 사용해야 한다.

In [2]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(1e-5)
X_train_sel = selector.fit_transform(X_train)
X_test_sel = selector.transform(X_test)
X_train_sel.shape

(23149, 14330)

In [11]:
# 4만7000개의 데이터 종류 중 1만4000개만 남음

In [9]:
# 베르누이 나이브베이즈 모델로 시험

In [3]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [4]:
%%time
model = BernoulliNB()
model.fit(X_train, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test))))

train accuracy:0.381
test accuracy :0.324
CPU times: user 23.6 s, sys: 4.05 s, total: 27.6 s
Wall time: 23.7 s


In [5]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

train accuracy:0.529
test accuracy :0.441
CPU times: user 19.8 s, sys: 3.23 s, total: 23 s
Wall time: 19.9 s


In [13]:
# 데이터의 종류 1/4만 썼을 때 테스트 퍼포먼스가 오히려 좋아졌다

# 나이브베이즈는 독립변수가 서로 독립적이라고 가정하고 있다
# 현실에서는 독립적이지 않으면 그 가정이 틀리기 때문에 퍼포먼스가 떨어지는 경우가 생긴다
# 독립변수 개수가 적어지면 그 가정이 더 잘 맞게 돼 오히려 퍼포먼스가 좋아지는 경우가 많이 발생한다
# 나이브베이즈는 텍스트에서 많이 쓰인다. 텍스트에서 중요한 것 중에 하나가 불용어. 불용어 통해서 필요없는 단어나 토큰을 걸러내는 게 퍼포먼스에 영향을 많이 준다

## 단일 변수 선택

In [15]:
# 모델을 갖다가 단일 변수 하나만 써보는 것
# 개별적으로 x를 하나씩 뽑아서 상관관계 구하거나 상호정보량(mutual information) 같은 비선형 상관관계를 구하거나 하는 게 밑의 방법들
# x 하나씩만 뽑아서 테스트 해보고 나서 그 중에서 가장 성능 좋은 것을 선택하는 것이기 때문에
# 개별적으로 성능이 나쁘지만 서로 다른 두 개를 합쳤을 때 성능이 좋아질 수 있다 이런건 선택이 안 된다. 그리디한 선택(처음에 할 때 제일 좋은 것)을 하게 된다

단일 변수 선택법은 각각의 독립변수를 하나만 사용한 예측모형의 성능을 이용하여 가장 분류성능 혹은 상관관계가 높은 변수만 선택하는 방법이다. 사이킷런 패키지의 feature_selection 서브패키지는 다음 성능지표를 제공한다.

* `chi2`: 카이제곱 검정 통계값
* `f_classif`: 분산분석(ANOVA) F검정 통계값
* `mutual_info_classif`: 상호정보량(mutual information)

하지만 단일 변수의 성능이 높은 특징만 모았을 때 전체 성능이 반드시 향상된다는 보장은 없다.

feature_selection 서브패키지는 성능이 좋은 변수만 사용하는 전처리기인 `SelectKBest` 클래스도 제공한다. 사용법은 다음과 같다.

In [6]:
from sklearn.feature_selection import chi2, SelectKBest

In [7]:
%%time

selector1 = SelectKBest(chi2, k=14330)
X_train1 = selector1.fit_transform(X_train, y_train)
X_test1 = selector1.transform(X_test)

model = BernoulliNB()
model.fit(X_train1, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train1))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test1))))

train accuracy:0.505
test accuracy :0.438
CPU times: user 19.8 s, sys: 4.15 s, total: 24 s
Wall time: 19.8 s


In [16]:
# 원래 성능보다 향상

## 다른 모형을 이용한 특성 중요도 계산

In [18]:
# 익스트렘 랜덤 포레스트를 사용하면 특성 중요도를 계산할 수 있었다
# 익스트림 랜덤 포레스트를 써서 모델을 선택하는 방법도 있다

특성 중요도(feature importance)를 계산할 수 있는 랜덤포레스트 등의 다른 모형을 사용하여 일단 특성을 선택하고 최종 분류는 다른 모형을 사용할 수도 있다.

In [8]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [9]:
%%time
n_sample = 10000
idx = np.random.choice(range(len(y_train)), n_sample)
model_sel = ExtraTreesClassifier(n_estimators=50).fit(X_train[idx, :], y_train[idx])
selector = SelectFromModel(model_sel, prefit=True, max_features=14330)
X_train_sel = selector.transform(X_train)
X_test_sel = selector.transform(X_test)

CPU times: user 26.7 s, sys: 710 ms, total: 27.4 s
Wall time: 26.9 s


In [20]:
# SelectFromModel이라는 전처리기 사용, 여기서 성능이 중요도를 구하고, 성능이 좋은 것을 14330개만 뽑게 해놨다

In [10]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

train accuracy:0.604
test accuracy :0.491
CPU times: user 18.7 s, sys: 3.73 s, total: 22.5 s
Wall time: 19.9 s


In [22]:
# 성능이 가장 좋게 나왔다
# 랜덤 포레스트 한 번 돌려서, 피처 임포턴스 뽑고, 뽑은 특성 중요도를 이용해 다시 베르누이 나이브베이즈를 했을 때 퍼포먼스가 좋아진 것을 확인할 수 있었다